<a href="https://colab.research.google.com/github/arkwith7/aSSIST_ML/blob/main/Llama2/Llama2_fine_tuning_korquad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama 2를 KorQuad 데이터셋에 맞게 fine-tuning 하기
### References:
1.   https://www.youtube.com/watch?v=LslC2nKEEGU
2.   https://colab.research.google.com/drive/1JDnGJbxT8fSqwnXY8J-XFo73AtiSuQMe?usp=sharing

In [1]:
!nvidia-smi

Wed Jan 24 01:10:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              44W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

## AutoTrain Advanced
https://github.com/huggingface/autotrain-advanced

In [2]:
!pip install -q autotrain-advanced

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.1/213.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 61.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 72.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━

## PyTorch 업데이트

In [3]:
!autotrain setup --update-torch

> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers
> INFO    Installing latest PyTorch
> INFO    Successfully installed latest PyTorch


## Korquad 데이터셋에 맞게 Llama2 Fine-Tuning
- 학습 데이터 디랙터리 "korquad_prompt_da"를 만들고 학습데이터 korquad_data_augmented.json를 디렉터리에 업로드
- autotrain 설정값: https://github.com/huggingface/autotrain-advanced/blob/f1367b590dfc53d240e9684779991da540590386/src/autotrain/cli/run_llm.py#L21 (**과거 버전[0.6.35]**)
- autotrain 설정값: https://github.com/huggingface/autotrain-advanced/blob/main/src/autotrain/cli/run_llm.py#L17 (**최신 버전[0.6.80]**)


In [4]:
# Data Augmentation 적용 x
# !autotrain llm --train \
#     --project_name "llama2-korquad-finetuning" \
#     --model "TinyPixel/Llama-2-7B-bf16-sharded" \
#     --data_path "korquad_prompt" \
#     --text_column "text" \
#     --use_peft \
#     --use_int4 \
#     --learning_rate 2e-4 \
#     --train_batch_size 4 \
#     --num_train_epochs 100 \
#     --trainer sft \
#     --model_max_length 256
# Data Augmentation 적용 o (과거 버전)[0.6.35]
# !autotrain llm --train \
#     --project_name "llama2-korquad-finetuning-da" \
#     --model "TinyPixel/Llama-2-7B-bf16-sharded" \
#     --data_path "korquad_prompt_da" \
#     --text_column "text" \
#     --use_peft \
#     --use_int4 \
#     --learning_rate 2e-4 \
#     --train_batch_size 8 \
#     --num_train_epochs 40 \
#     --trainer sft \
#     --model_max_length 256
# Data Augmentation 적용 o (최신 버전)[0.6.80]
!autotrain llm --train \
    --project-name "llama2-korquad-finetuning-da" \
    --model "TinyPixel/Llama-2-7B-bf16-sharded" \
    --data-path "korquad_prompt_da" \
    --text-column "text" \
    --peft \
    --quantization "int4" \
    --lr 2e-4 \
    --batch-size 8 \
    --epochs 40 \
    --trainer sft \
    --model_max_length 256

> INFO    Running LLM
> INFO    Params: Namespace(version=False, text_column='text', rejected_text_column='rejected', prompt_text_column='prompt', model_ref=None, warmup_ratio=0.1, optimizer='adamw_torch', scheduler='linear', weight_decay=0.0, max_grad_norm=1.0, add_eos_token=False, block_size=-1, peft=True, lora_r=16, lora_alpha=32, lora_dropout=0.05, logging_steps=-1, evaluation_strategy='epoch', save_total_limit=1, save_strategy='epoch', auto_find_batch_size=False, mixed_precision=None, quantization='int4', model_max_length=256, trainer='sft', target_modules=None, merge_adapter=False, use_flash_attention_2=False, dpo_beta=0.1, apply_chat_template=False, padding=None, train=True, deploy=False, inference=False, username=None, backend='local-cli', token=None, repo_id=None, push_to_hub=False, model='TinyPixel/Llama-2-7B-bf16-sharded', project_name='llama2-korquad-finetuning-da', seed=42, epochs=40, gradient_accumulation=1, disable_gradient_checkpointing=False, lr=0.0002, log='none', dat

# 학습결과 zip 파일로 압축후 다운로드

In [5]:
import zipfile
import shutil
from google.colab import files

# 압축할 폴더 이름
#folder_name = "llama2-korquad-finetuning"    # Data Augmentation 적용 x
folder_name = "llama2-korquad-finetuning-da"  # Data Augmentation 적용 o

# 생성될 ZIP 파일 이름
#zip_file_name = "llama2-korquad-finetuning.zip"  # Data Augmentation 적용 x
zip_file_name = "llama2-korquad-finetuning-da.zip" # Data Augmentation 적용 o

# 폴더를 ZIP 파일로 압축
shutil.make_archive(zip_file_name[:-4], 'zip', folder_name)

# ZIP 파일을 로컬로 다운로드
files.download(zip_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>